In [1]:
from w5d1_solutions import celeb_DCGAN, celeb_mini_DCGAN
import w5d1_utils
from GAN_implementation.models import Generator, initialize_weights_generator

my_Generator = Generator(100, 64, 3, 512, 4)
initialize_weights_generator(my_Generator)
w5d1_utils.print_param_count(my_Generator, celeb_mini_DCGAN.netG)

/Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: <D9493EF5-8DAB-3A5D-85D5-684F04544B84> /Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <52337463-FA04-3610-8715-68C586545714> /Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


Model 1, total params = 3576704
Model 2, total params = 3576704
All parameter counts match!


,name_1,shape_1,num_params_1,num_params_2,shape_2,name_2
0,project_and_reshape.0.weight,"(8192, 100)",819200,819200,"(8192, 100)",project_and_reshape.0.weight
1,project_and_reshape.2.weight,"(512,)",512,512,"(512,)",project_and_reshape.2.weight
2,project_and_reshape.2.bias,"(512,)",512,512,"(512,)",project_and_reshape.2.bias
3,layers.0.conv.weight,"(512, 256, 4, 4)",2097152,2097152,"(512, 256, 4, 4)",layers.0.0.weight
4,layers.0.bn.weight,"(256,)",256,256,"(256,)",layers.0.1.weight
5,layers.0.bn.bias,"(256,)",256,256,"(256,)",layers.0.1.bias
6,layers.1.conv.weight,"(256, 128, 4, 4)",524288,524288,"(256, 128, 4, 4)",layers.1.0.weight
7,layers.1.bn.weight,"(128,)",128,128,"(128,)",layers.1.1.weight
8,layers.1.bn.bias,"(128,)",128,128,"(128,)",layers.1.1.bias
9,layers.2.conv.weight,"(128, 64, 4, 4)",131072,131072,"(128, 64, 4, 4)",layers.2.0.weight


In [2]:
from GAN_implementation.models import Discriminator
my_Discriminator = Discriminator(64, 3, 512, 4)
w5d1_utils.print_param_count(my_Discriminator, celeb_mini_DCGAN.netD)

Model 1, total params = 2766528
Model 2, total params = 2766528
All parameter counts match!


,name_1,shape_1,num_params_1,num_params_2,shape_2,name_2
0,layers.0.conv.weight,"(64, 3, 4, 4)",3072,3072,"(64, 3, 4, 4)",layers.0.0.weight
1,layers.0.conv.bias,"(64,)",64,64,"(64,)",layers.0.0.bias
2,layers.1.conv.weight,"(128, 64, 4, 4)",131072,131072,"(128, 64, 4, 4)",layers.1.0.weight
3,layers.1.conv.bias,"(128,)",128,128,"(128,)",layers.1.0.bias
4,layers.1.bn.weight,"(128,)",128,128,"(128,)",layers.1.1.weight
5,layers.1.bn.bias,"(128,)",128,128,"(128,)",layers.1.1.bias
6,layers.2.conv.weight,"(256, 128, 4, 4)",524288,524288,"(256, 128, 4, 4)",layers.2.0.weight
7,layers.2.conv.bias,"(256,)",256,256,"(256,)",layers.2.0.bias
8,layers.2.bn.weight,"(256,)",256,256,"(256,)",layers.2.1.weight
9,layers.2.bn.bias,"(256,)",256,256,"(256,)",layers.2.1.bias


In [5]:
import torch.nn as nn
from GAN_implementation.layers import Conv2d, ConvTranspose2d
def initialize_weights_discriminator(model: nn.Module) -> None:
    for i in model.modules():
        if isinstance(i, Conv2d):
            nn.init.normal_(i.weight, 0.0, 0.02)
            if i.bias is not None:
                nn.init.constant_(i.bias, 0)
        elif isinstance(i, nn.BatchNorm2d):
            nn.init.normal_(i.weight, 1.0, 0.02)
            nn.init.constant_(i.bias, 0)
        

initialize_weights_discriminator(my_Discriminator)

import torch.nn as nn
def initialize_weights_generator(model) -> None:
    for i in model.modules():
        if isinstance(i, ConvTranspose2d):
            nn.init.normal_(i.weight, 0.0, 0.02)
            if i.bias is not None:
                nn.init.constant_(i.bias, 0)
        elif isinstance(i, nn.BatchNorm2d):
            nn.init.normal_(i.weight, 1.0, 0.02)
            nn.init.constant_(i.bias, 0)

initialize_weights_generator(my_Generator)

In [75]:
from typing import Optional, Union
from torch import optim
import time 
import torch as t
from tqdm.notebook import tqdm
import wandb

def train_generator_discriminator(
    netG: Generator, 
    netD: Discriminator, 
    optG,
    optD,
    trainloader,
    epochs: int,
    max_epoch_duration: Optional[Union[int, float]] = None,           # Each epoch terminates after this many seconds
    log_netG_output_interval: Optional[Union[int, float]] = None,     # Generator output is logged at this frequency
    use_wandb: bool = True,
    device: str = "cpu"):

    criterion = nn.BCELoss()
    fixed_noise = t.randn(64, 100, device=device)

    start_time = time.time()

    if use_wandb:
        wandb.init(project="GAN_implementation", entity="arena-ldn")
        wandb.watch(netG, log="all")
        wandb.watch(netD, log="all")

    
    for epoch in range(epochs):
        pbar = tqdm(enumerate(trainloader, 0), total=len(trainloader))
        for i, data in pbar:
            # 1. Train the discriminator on real+fake
            netD.zero_grad()
            # 1A. Train on real
            real_images = data[0]
            batch_size = real_images.size(0)
            label = t.full((batch_size,), 1, dtype=t.float32)
            output = netD(real_images)
            errD_real = criterion(output.flatten(), label)
            errD_real.backward()
            D_x = output.mean().item()

            # 1B. Train on fake
            noise = t.randn(batch_size, 100, device=device)
            fake = netG(noise)
            label.fill_(0)
            output = netD(fake.detach())
            errD_fake = criterion(output.flatten(), label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optD.step()

            # 2. Train the generator on the discriminator's error
            netG.zero_grad()
            label.fill_(1)
            output = netD(fake)
            errG = criterion(output.flatten(), label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optG.step()

            # 3. Log metrics
            pbar.set_description(f"Epoch: {epoch}, Iteration: {i}, D(x): {D_x:.2f}, D(G(z)): {D_G_z1:.2f}/{D_G_z2:.2f}, D loss: {errD.item():.2f}, G loss: {errG.item():.2f}")

            if i % log_netG_output_interval == 0:
                # print(f"Epoch: {epoch}, Iteration: {i}, D(x): {D_x:.2f}, D(G(z)): {D_G_z1:.2f}/{D_G_z2:.2f}, D loss: {errD.item():.2f}, G loss: {errG.item():.2f}")
                if use_wandb:
                    wandb.log({
                        "D(x)": D_x,
                        "D(G(z))": D_G_z1,
                        "D(G(z))": D_G_z2,
                        "D loss": errD.item(),
                        "G loss": errG.item()
                    })
            
            if use_wandb and (i % 100 == 0):
                with t.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                wandb.log({"images": [wandb.Image(x, caption="epoch: " + str(epoch) + " iteration: " + str(i)) for x in fake]})
            
            if max_epoch_duration is not None:
                if time.time() - start_time > max_epoch_duration:
                    break

        print("Training complete")

    return netG, netD

from GAN_implementation.models import Generator
my_Generator = Generator(100, 64, 3, 512, 4)
initialize_weights_generator(my_Generator)
my_Discriminator = Discriminator(64, 3, 512, 4)
initialize_weights_discriminator(my_Discriminator)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
discriminator_optimizer = optim.Adam(my_Discriminator.parameters(), lr=0.0002, betas=(0.5,0.999))
generator_optimizer = optim.Adam(my_Generator.parameters(), lr=0.0002, betas=(0.5,0.999))

my_Generator, my_Discriminator = train_generator_discriminator(
    my_Generator,
    my_Discriminator,
    generator_optimizer,
    discriminator_optimizer,
    trainloader,
    epochs=1,
    max_epoch_duration=60*60*2,
    log_netG_output_interval=100,
    use_wandb=False,
    device="cpu",
)
    

  0%|          | 0/12663 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [77]:
my_Generator.latent_dim_size

100

In [64]:
my_Generator(t.randn(100, 100)).shape

torch.Size([100, 3, 64, 64])

date and time: 12/06/2022, 12:12:35


In [81]:
noise = t.randn(100, 100)
real_images = next(iter(trainloader))[0]
D_x = my_Discriminator(real_images).mean()
fake = my_Generator(noise)
D_G_z = my_Discriminator(fake.detach()).mean()

(t.log(D_x).mean() + t.log(1 - D_G_z).mean())

tensor(-7.0334e-06, grad_fn=<AddBackward0>)

In [83]:
#Binary Cross Entropy Loss
import torch
torch.nn.BCELoss()(D_x,)

TypeError: BCELoss.forward() missing 1 required positional argument: 'target'

In [84]:
trainloader.batch_size


16